In [143]:
import requests
import re # Regular Expression parsing을 위함
import csv # csv로 내보내기 위함
from bs4 import BeautifulSoup
from time import sleep # Selenium을 사용하면 브라우저 로딩시간을 기다려야할 필요가 생김. 잠깐 멈추기  위함
from selenium import webdriver # Webdriver를 사용해서 Chrome 브라우저를 컨트롤하기 위함
from selenium.common.exceptions import NoSuchElementException # 없는 요소 찾을 때 오류 처리를 위함
from datetime import datetime # 오늘 연월일 사용을 위함
from IPython.display import clear_output # Cell output clear를 위함

############# 필요한 기본 변수들, 드라이버 설정 진행 ############
now = datetime.now() #현재 시간을 받는다. 연월일을 활용하기 위함
driver = webdriver.Chrome('/Users/youngjinlim/Coding/Drivers/chromedriver_mac64/chromedriver')
driver.set_window_size(1280, 720) # 크롬 창 크기 조절
#크롬 구동을 위한 크롬드라이버 사용. PATH 등록을 해 주어야 하는데, 다음 링크를 참고하자. https://www.kenst.com/2015/03/including-the-chromedriver-location-in-macos-system-path/
########################################################

In [134]:
def making_url():
    # Verge URL 규칙에 따라 URL 생성
    target_page_list = []
    for each_year in range(2011,now.year+1):
        month_start = 1
        month_end = 13
        if each_year == now.year:
            month_end = now.month + 1
        elif each_year == 2011:
            month_start = 4
        for each_month in range(month_start,month_end):
            url = "https://www.theverge.com/archives/tech/" + str(each_year) + "/" + str(each_month)
            target_page_list.append(url)
    return target_page_list

In [144]:
def archieve_page_mining(target_page_list):
    #기사 모여있는 URL 리스트를 가지고 기사 URL 모으기
    total_article_link_list = []
    for each_url in target_page_list:
        driver.get(each_url)
        try: #Load More 버튼이 안 나올때까지 무한으로 버튼을 눌러준다.
            while True:
                driver.find_element_by_class_name('p-button c-archives-load-more__button').click()
                time(1.2) #로딩 기다림
        except NoSuchElementException:
            #더 이상 버튼이 안 나오면, 모두 확장한 것이므로 기사 URL을 수집한다.
            articlelink_list = driver.find_elements_by_css_selector('a[data-analytics-link][href]')
            # 기사 제목 링크의 구조가 'a[data-analytics-link][href]' 이 CSS selector구분으로 선택 가능하게 되어있음
            for each in articlelink_list:
                each_link = each.get_attribute('href') #href속성 : 링크 가져옴
                #일부 동일한 CSS Selector 구조를 가지면서 이상한 링크인 녀석이 숨어 있음
                #기사 링크만 뽑아내기 위해, 아래와 같은 구조 사용
                if re.match(re.compile(r'https://www.theverge.com/20'),each_link) != None:
                    #기사 URL이면 리스트에 넣음
                    total_article_link_list.append(each_link)
    return total_article_link_list

In [136]:
def html_parsing_get_contents(total_article_link_list):
    #URL에서 HTML 코드를 받은 후, 파싱하여 제목 / 작성날짜 / 기사내용 으로 분리하여 저장함
    parsed_article_list = []
    no_title_list = [] 
    no_contents_list = []
    no_date_list = []
    
    # 프로그레스 표시용 변수
    length = len(total_article_link_list)
    progress = 1
    errnum_title = 0
    errnum_date = 0
    errnum_contents = 0
    
    for each_link in total_article_link_list:
        req = requests.get(each_link)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')
        
        try:
            title_pa = soup.find('h1', class_='c-page-title').getText() #내부 텍스트만 가져오기 위한 getText()
        except AttributeError :# 발견 못하면
            try:
                title_pa = soup.find('h1', class_='instapaper_title').getText()
                # 다른 유형의 기사가 있었음
            except : # 진짜 못찾으면
                title_pa = 'None'
                print('No title detected')
                no_title_list.append(each_link)
                errnum_title += 1

        try:
            # URL에 작성일자가 나와있어서 이걸 가져오기로 한다.
            link_sliced = each_link.lstrip('https://www.theverge.com/') # 링크 앞의 사이트 이름 자름
            delimiter = "/" # 자른 문자열에서, /가 3번째 오는 곳 까지 자르면 그게 날짜임
            endposition_date= [pos for pos, char in enumerate(link_sliced) if char == delimiter][2] # / 세 번째 위치 찾음
            published_time_pa = link_sliced[0:endposition_date] # / 3번째 위치까지 자름
            # published_time_pa = soup.find('time')['datetime'] #기사 작성시간 잘 찾아옴. ['datetime'] : datetime attribute를 가져오기 위한 argument
        except: # 발견 못하면
            published_time_pa = "None"
            print('No date detected')
            no_date_list.append(each_link)
            errnum_date += 1
            
        try:
            contents_pa = soup.find('div', class_='c-entry-content').getText() # 기사 태그 내부에서 텍스트만 가져옴.
        except AttributeError: # 발견 못하면
            try : 
                contents_pa = soup.find('div', class_='m-feature').getText()
                # 다른 유형의 기사가 있었음
            except: # 진짜 못찾으면
                contents_pa = "None"
                print('No contents detected')
                no_contents_list.append(each_link)
                errnum_contents += 1
            
        article = dict(title = title_pa, time = published_time_pa, contents = contents_pa, link = each_link) #딕셔너리로 저장 
        parsed_article_list.append(article) # 딕셔너리 객체 하나를 기사 하나로 보고, 리스트에 저장함
        clear_output(wait=True) # IPython 출력부분 클리어. Progress 체크를 위함.
        print(f'{length}, 완료됨:{progress}, 제목없음:{errnum_title}, 내용없음:{errnum_contents}, 날짜없음:{errnum_date}') # Progress 체크부분
        progress += 1
        
    return ({'parsed_article_list':parsed_article_list, 'notitle_url_list':no_title_list,
                'nodate_url_list':no_date_list, 'nocontents_url_list':no_contents_list})

In [137]:
# 받은 리스트를 TXT로 내보내는 함수
def write_to_txt(some_list, filename):
    with open (filename + '.txt', 'wt', encoding='utf-8') as f:
        for each in some_list:
            f.write(str(each)+'\n')

In [138]:
# 긁어온 기사를 CSV로 내보내는 함수
def write_to_csv(article_list, filename):
    with open (filename + '.csv', 'wt', encoding='utf-8') as f:
        wr = csv.writer(f)
        for each in article_list:
            wr.writerow([each['time'], each['title'], each['contents']])

## 함수 실행 영역

In [145]:
# 기사 URL 수집 및 txt로 내보냄 
target_page_list = making_url()
target_article_link_list = archieve_page_mining(target_page_list)
write_to_txt('url_list', target_article_link_list) 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=78.0.3904.70)


In [146]:
#수집 및 파싱
article_list = html_parsing_get_contents(target_article_link_list)

2070, 완료됨:37, 제목없음:0, 내용없음:0, 날짜없음:0


KeyboardInterrupt: 

In [140]:
#기사들을 csv로 내보냄
write_to_csv(article_list['parsed_article_list'],'parsed_article')

In [141]:
#날짜, 제목, 내용 없는 링크들을 내보냄
write_to_txt(article_list['nodate_url_list'],'nodate_link')
write_to_txt(article_list['notitle_url_list'],'notitle_link',)
write_to_txt(article_list['nocontents_url_list'],'nocontents_link')